In [ ]:
import os
import requests
import os
import sys
from tqdm import tqdm
import time
import chat
import numpy as np

# More secure way
incubator_endpoint = "https://eyqincubator.america.fabric.ey.com/eyq/canadaeast/api/"
incubator_key = "toNx9GPyvXRB9C6sDC8G47XAO2IKGeZV"

In [ ]:
def completion(prompt):
    
    api_version = "2023-05-15"
    
    model = "gpt-4-turbo"  # Replace with desired modelS
    
    headers = {
        "x-api-key": incubator_key
    }
    
    query_params = {
        "api-version": api_version
    }
    
    
    body = {
        "messages":[
            {"role": "system", "content": "You're an expert software engineer specialized in Cobol and PySpark programming languages. your responsibility is translate Cobol codes into PySpark accurately and including all Proccess."},
            {"role":"user","content": prompt}
        ]
    }
    
    full_path = incubator_endpoint + "/openai/deployments/" + model + "/chat/completions"
    
    response = requests.post(full_path, json=body, headers=headers, params=query_params)
    
    return response

In [ ]:
with open('DWSD0612_completo_V3.txt', 'r') as file:
    data = file.read().strip()
    data = ' '.join(data.split())

In [ ]:
final = []
divisions = ["ENVIRONMENT DIVISION", "DATA DIVISION","PROCEDURE DIVISION"]
data_split = data.split('DIVISION')[2:]
for i,row in enumerate(data_split):
    final.append(f'{divisions[i]} {row}')


In [ ]:
# responses = []
# for code in tqdm(final):
#     time.sleep(1)
#     response = completion(prompt=f"""
#     Given the following Cobol code, translate it into PySpark code:

#     {code}
            
#     Ensure that the PySpark code includes all the necessary processes.
#     Return Only the Pyspark Code
#     """)
#     try:
#         responses.append(response.json())
#     except:
#         responses.append(response.text)

usage = []
responses = []
for code in tqdm(final):
    response = chat.generate(prompt=f"""Given the following COBOL code snippet. Your role will be to decipher the COBOL code, create an equivalent PySpark code with the same functionality, including all the necessary processes.

COBOL code:

```
{code}
```

Your task is to do the following steps:

1. Analyze and comprehend the COBOL code snippet above.
2. Write an equivalent PySpark code.
3. The equivalent code should provide the exact same output as the COBOL code.
4. Follow best practices while crafting your code, like writing clean, well-indented, efficient, and concise code.
6. Return only the PySpark code in the response.""",temperature=0.1)
    responses.append(response)
    usage.append(((response.usage.prompt_tokens/1000) * 0.006)+((response.usage.completion_tokens/1000) * 0.012))
print(np.sum(usage))

In [ ]:
# import re

# sparks = []
# for resp in  responses:

#     code = re.findall(r'```python\n(.*?)```', resp.choices[0].message.content, re.DOTALL)[0]
#     sparks.append(f'```python\n {code} ```') 
# final_sparks = '\n\n\n '.join(sparks)

In [ ]:
final_sparks = '\n\n'.join(resp.choices[0].message.content for resp in responses)

In [ ]:
for resp in  responses:
    with open("Outputs/Cobol2Spark_teste_34.md", "w") as text_file:
        text_file.write(final_sparks)

In [ ]:
final_code = chat.generate(prompt=f"""Given the following PySpark code snippet. Your role will be to decipher the code, rearrange the code and return the PySpark code focusing in Clean code, well-indented, efficient, and concise code.
                            

Pyspark code:

```
{final_sparks}
```

Your task is to:

1. Analyze and comprehend the PySpark code snippet above.
2. Clean the code, make it well-indented, efficient, and concise code.
3. Follow best practices while crafting your code, like writing clean, well-indented, efficient, and concise code.
6. Return only the PySpark code.""",temperature=0.7)

print(((response.usage.prompt_tokens/1000) * 0.006)+((response.usage.completion_tokens/1000) * 0.012))

In [ ]:
for resp in  responses:
    with open("Outputs/Cobol2Spark_teste_33.md", "w") as text_file:
        text_file.write(final_code.choices[0].message.content)